In [85]:
import pandas as pd 

In [86]:
import pandas as pd
import numpy as np

pd.options.display.float_format = "{:.2f}".format
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score

original_data_filtered = pd.read_parquet('data/original_data_filtered.parquet')
original_data_filtered

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период погашения дебиторской задолженности, дни","Период погашения кредиторской задолженности, дни","Рентабельность активов (ROA), %","Рентабельность затрат, %","Рентабельность капитала (ROE), %","Рентабельность продаж, %","Себестоимость продаж, RUB","Собственный оборотный капитал, RUB","Соотношение чистого долга к капиталу, %",Категориальная переменная
0,0,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,49.00,48.21,0.70,0.16,2.42,0.14,2775000.00,342000.00,-0.15,False
1,1,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,93.00,93.43,-0.28,0.02,-1.04,0.02,2096000.00,98000.00,-0.27,False
2,2,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,80.00,144.05,0.03,0.05,0.22,0.05,1699000.00,121000.00,-3.15,False
3,3,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,23.00,29.73,0.57,0.08,1.77,0.08,26989000.00,1954000.00,-0.15,False
4,4,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,147.00,173.59,0.02,0.01,0.07,0.01,8440000.00,2205000.00,-0.05,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77276,77276,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,99.00,270.34,0.01,-0.07,0.51,-0.07,538535000.00,-106245000.00,-5.68,False
77277,77277,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,82.00,180.63,0.01,0.02,1.32,0.02,475278000.00,-26689000.00,-56.77,False
77278,77278,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,51.00,75.47,0.00,0.00,0.84,0.00,572141000.00,24473000.00,2.79,False
77279,77279,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,78.00,68.78,0.04,-0.03,1.19,-0.03,275635000.00,25007000.00,-4.58,False


In [87]:
bankrupt_data = original_data_filtered[original_data_filtered['Категориальная переменная'] == True]
no_bankrupt_data = original_data_filtered[original_data_filtered['Категориальная переменная'] == False]

bankrupt_data['Год иска'] = bankrupt_data['Дата иска'].apply(lambda x: x.year - 2 if x.month > 3 else x.year - 1)
bankrupt_data['Год'] = bankrupt_data['Год'].astype(int)
no_bankrupt_data['Год'] = no_bankrupt_data['Год'].astype(int)

filtered_data = bankrupt_data[bankrupt_data['Год'] == bankrupt_data['Год иска']]

filtered_data

C:\Users\User\AppData\Local\Temp\ipykernel_28228\3464714820.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankrupt_data['Год иска'] = bankrupt_data['Дата иска'].apply(lambda x: x.year - 2 if x.month > 3 else x.year - 1)
C:\Users\User\AppData\Local\Temp\ipykernel_28228\3464714820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankrupt_data['Год'] = bankrupt_data['Год'].astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_28228\3464714820.py:6: SettingWithCopyWarning: 
A value is trying to be set

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период погашения кредиторской задолженности, дни","Рентабельность активов (ROA), %","Рентабельность затрат, %","Рентабельность капитала (ROE), %","Рентабельность продаж, %","Себестоимость продаж, RUB","Собственный оборотный капитал, RUB","Соотношение чистого долга к капиталу, %",Категориальная переменная,Год иска
16,16,"1015 УСМР, АО",1027739381658.00,"143003, Московская обл, Одинцово г, г.о. Одинц...",7734008581.00,Московская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (13.07.20...,"""STR""",2017-01-24,...,297.01,0.00,0.00,0.10,0.00,2345886000.00,-202179000.00,-2.89,True,2016
24,24,"162-КЖИ, ООО",1143668025027.00,"123308, г. Москва, вн.тер.г. муниципальный окр...",3664135233.00,Москва,Строительство жилых и нежилых зданий,Регистрирующим органом принято решение о предс...,"""STR""",2016-03-30,...,NaN,NaN,NaN,NaN,NaN,NaN,7720000.00,7031.40,True,2015
59,59,"24 ЭТАЖ, ООО",1105027014971.00,"119361, г. Москва, ул. Озёрная, д. 42 пом. I э...",5027169331.00,Москва,Строительство жилых и нежилых зданий,Ликвидировано вследствие банкротства (02.08.20...,"""STR""",2020-06-19,...,NaN,NaN,0.04,NaN,0.04,849137000.00,4967000.00,0.20,True,2018
80,80,"2МЕН ГРУПП, АО",1067746424899.00,"625002, Тюменская обл., г. Тюмень, ул. Комсомо...",7701651356.00,Тюменская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (03.08.20...,"""STR""",2019-10-21,...,3057.84,0.00,0.31,0.02,0.24,43552000.00,-285582000.00,7.75,True,2017
115,115,"534 УНР, АО",1037739897205.00,"141013, Московская обл, г.о. Мытищи, г Мытищи,...",7743509143.00,Московская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2020-04-29,...,68.83,0.13,0.04,0.40,0.03,5669470000.00,267428000.00,-0.00,True,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77173,77173,"ЯЛТА, ООО",1037832006706.00,"197136, г. Санкт-Петербург, ул. Профессора Поп...",7814127961.00,Санкт-Петербург,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2017-03-17,...,273.27,-0.00,-0.10,-0.04,-0.11,41533000.00,15188000.00,8.48,True,2016
77225,77225,"ЯНТАРЬСЕРВИСБАЛТИК, ООО",1103925022948.00,", Калининградская обл, г.о. город Калининград,...",3906221283.00,Калининградская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2017-02-07,...,200.09,0.00,0.00,0.34,0.00,344484000.00,-64195000.00,0.47,True,2016
77236,77236,"ЯРАВТОДОР, ООО",1147602008520.00,"150506, Ярославская обл., Ярославский район, п...",7602110885.00,Ярославская область,Строительство жилых и нежилых зданий,\nСообщение о признаках банкротства (17.10.202...,"""STR""",2020-11-02,...,724.22,0.01,0.06,0.17,0.06,228319000.00,-12804000.00,1.42,True,2018
77249,77249,"ЯРДА, ООО",1186658056903.00,"620062, Свердловская обл., г. Екатеринбург, ул...",6670470764.00,Свердловская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (23.08.20...,"""STR""",2021-08-09,...,146.07,-0.08,-0.02,0.30,-0.02,149749000.00,-16790000.00,1.69,True,2019


In [103]:
filtered_data = filtered_data.dropna()
no_bankrupt_data = no_bankrupt_data.dropna()
no_bankrupt_data

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период погашения дебиторской задолженности, дни","Период погашения кредиторской задолженности, дни","Рентабельность активов (ROA), %","Рентабельность затрат, %","Рентабельность капитала (ROE), %","Рентабельность продаж, %","Себестоимость продаж, RUB","Собственный оборотный капитал, RUB","Соотношение чистого долга к капиталу, %",Категориальная переменная
0,0,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,49.00,48.21,0.70,0.16,2.42,0.14,2775000.00,342000.00,-0.15,False
1,1,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,93.00,93.43,-0.28,0.02,-1.04,0.02,2096000.00,98000.00,-0.27,False
2,2,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,80.00,144.05,0.03,0.05,0.22,0.05,1699000.00,121000.00,-3.15,False
40,40,"1МСМУ СТАЛЬМОНТАЖ, ООО",1077760036188.00,"129329, г. Москва, вн.тер.г. муниципальный окр...",7705808840.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,64.00,79.43,0.05,0.01,4.21,0.01,655328000.00,-23944000.00,-3.55,False
42,42,"1МСМУ СТАЛЬМОНТАЖ, ООО",1077760036188.00,"129329, г. Москва, вн.тер.г. муниципальный окр...",7705808840.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,450.00,487.88,0.01,0.01,0.38,0.01,317055000.00,7233000.00,-0.85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77274,77274,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,275.00,747.46,0.00,0.04,0.02,0.03,78561000.00,-153116000.00,-0.40,False
77276,77276,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,99.00,270.34,0.01,-0.07,0.51,-0.07,538535000.00,-106245000.00,-5.68,False
77277,77277,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,82.00,180.63,0.01,0.02,1.32,0.02,475278000.00,-26689000.00,-56.77,False
77278,77278,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,51.00,75.47,0.00,0.00,0.84,0.00,572141000.00,24473000.00,2.79,False


In [112]:
no_bankrupt_data

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период погашения дебиторской задолженности, дни","Период погашения кредиторской задолженности, дни","Рентабельность активов (ROA), %","Рентабельность затрат, %","Рентабельность капитала (ROE), %","Рентабельность продаж, %","Себестоимость продаж, RUB","Собственный оборотный капитал, RUB","Соотношение чистого долга к капиталу, %",Категориальная переменная
0,0,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,49.00,48.21,0.70,0.16,2.42,0.14,2775000.00,342000.00,-0.15,False
1,1,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,93.00,93.43,-0.28,0.02,-1.04,0.02,2096000.00,98000.00,-0.27,False
2,2,"100РА ЛТД, ООО",1135038000679.00,"119121, г. Москва, вн.тер.г. муниципальный окр...",5038096300.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,80.00,144.05,0.03,0.05,0.22,0.05,1699000.00,121000.00,-3.15,False
40,40,"1МСМУ СТАЛЬМОНТАЖ, ООО",1077760036188.00,"129329, г. Москва, вн.тер.г. муниципальный окр...",7705808840.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,64.00,79.43,0.05,0.01,4.21,0.01,655328000.00,-23944000.00,-3.55,False
42,42,"1МСМУ СТАЛЬМОНТАЖ, ООО",1077760036188.00,"129329, г. Москва, вн.тер.г. муниципальный окр...",7705808840.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,450.00,487.88,0.01,0.01,0.38,0.01,317055000.00,7233000.00,-0.85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77274,77274,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,275.00,747.46,0.00,0.04,0.02,0.03,78561000.00,-153116000.00,-0.40,False
77276,77276,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,99.00,270.34,0.01,-0.07,0.51,-0.07,538535000.00,-106245000.00,-5.68,False
77277,77277,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,82.00,180.63,0.01,0.02,1.32,0.02,475278000.00,-26689000.00,-56.77,False
77278,77278,"ЯСУ, ООО",1137604013172.00,"150040, Ярославская обл, г.о. город Ярославль,...",7604248060.00,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,51.00,75.47,0.00,0.00,0.84,0.00,572141000.00,24473000.00,2.79,False


In [120]:
import pandas as pd

# Предположим, что у вас уже есть dataframe с оригинальными данными original_data_filtered

# ... (ваш код для фильтрации и обработки данных)

# Создаем dataframe с парами компаний
pairs = []
used_companies = set()

for index, row in filtered_data.iterrows():
    year = row['Год']
    assets = row[' Активы  всего, RUB']  # Предположим, что у вас есть столбец 'Активы всего, RUB'

    # Фильтруем данные не банкротов для текущего года
    candidates = no_bankrupt_data[(no_bankrupt_data['Год'] == year)]

    if not candidates.empty:
        # Находим ближайшую компанию по размеру активов
        nearest_index = (candidates[' Активы  всего, RUB'] - assets).abs().idxmin()
        nearest_company_name = no_bankrupt_data.loc[nearest_index, 'Наименование']
        # Проверяем, использовали ли уже компанию в предыдущих парах
        if nearest_company_name not in used_companies:
            # Расчитываем разницу в процентах между банкротной и ближайшей компаниями
            bankrupt_assets = filtered_data.at[index, ' Активы  всего, RUB']
            nearest_assets = no_bankrupt_data.at[nearest_index, ' Активы  всего, RUB']
            percent_difference = ((nearest_assets - bankrupt_assets) / bankrupt_assets) * 100

            # Создаем пару и добавляем в список
            pair = {
                'Банкротная компания ID': index,
                'Ближайшая компания ID': nearest_index,
                'Год': year,
                'Разница в %': percent_difference,
            }
            pairs.append(pair)

            # Добавляем компанию в использованные
            used_companies.add(nearest_company_name)

# Создаем dataframe из списка пар
pairs_df = pd.DataFrame(pairs)

In [121]:
pairs_df

,Банкротная компания ID,Ближайшая компания ID,Год,Разница в %
0,16,39123,2016,0.09
1,80,41643,2017,-1.03
2,130,44832,2018,-0.07
3,172,40177,2018,-0.03
4,237,50733,2015,-0.01
...,...,...,...,...
643,76909,23461,2018,-0.29
644,77085,6378,2019,0.04
645,77150,21796,2019,0.00
646,77236,35301,2018,-0.01


In [122]:

# Извлекаем строки из оригинальных датафреймов по индексам
bankrupt_companies = filtered_data.loc[pairs_df['Банкротная компания ID']]
nearest_companies = no_bankrupt_data.loc[pairs_df['Ближайшая компания ID']]

In [123]:
all_data = pd.concat([nearest_companies, bankrupt_companies])
all_data = all_data.drop_duplicates(subset='Наименование',keep=False)
all_data

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период погашения кредиторской задолженности, дни","Рентабельность активов (ROA), %","Рентабельность затрат, %","Рентабельность капитала (ROE), %","Рентабельность продаж, %","Себестоимость продаж, RUB","Собственный оборотный капитал, RUB","Соотношение чистого долга к капиталу, %",Категориальная переменная,Год иска
39123,39123,"ПЭТРУСКО, ООО",1137746210854.00,"119285, г. Москва, вн.тер.г. муниципальный окр...",7730682729.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,331.78,0.03,0.01,0.51,0.01,1865839000.00,-486123000.00,1.33,False,NaN
41643,41643,"РОССТРОЙИНВЕСТ, ЗАО",1037828003971.00,"197198, г. Санкт-Петербург, проспект Добролюбо...",7813161127.00,Санкт-Петербург,Строительство жилых и нежилых зданий,Прекратило деятельность при слиянии (07.12.2022).,"""STR""",1970-01-01,...,1666.38,0.00,0.18,0.00,0.15,77752000.00,483423000.00,-0.02,False,NaN
44832,44832,"СЕРВИССТРОЙ, ООО",1178617024740.00,"628680, Ханты-Мансийский Автономный Округ - Юг...",8605029850.00,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,53.83,-0.03,0.08,-0.12,0.07,350368000.00,9225000.00,-0.90,False,NaN
40177,40177,"РЕЛЬЕФ, ООО",1127847464986.00,"195299, г. Санкт-Петербург, проспект Гражданск...",7804492588.00,Санкт-Петербург,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,193.02,0.02,0.03,0.39,0.03,645637000.00,-55598000.00,-0.90,False,NaN
50733,50733,"СК-СТРОЙ, ООО",1085405016289.00,"630132, Новосибирская обл, г.о. город Новосиби...",5405375060.00,Новосибирская область,Строительство жилых и нежилых зданий,\n\n\n,"""А1"", ""STR""",1970-01-01,...,145.50,0.01,0.01,0.05,0.01,446923000.00,-30233000.00,-0.32,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76909,76909,"ЮНИТЕЛ ГРУПП, ООО",5067746007335.00,"108811, г. Москва, км. Киевское Шоссе 22-Й (П ...",7727584778.00,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2020-05-06,...,387.56,-0.00,-0.07,-0.02,-0.07,279664000.00,23409000.00,0.02,True,2018.00
77085,77085,"ЮТА, ООО СК",1138617000026.00,"628452, Ханты-Мансийский Автономный Округ - Юг...",8617031109.00,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2021-05-17,...,311.34,0.00,0.02,0.08,0.02,283636000.00,-50660000.00,-0.11,True,2019.00
77150,77150,"ЯИК, ООО",1125027002682.00,"140011, Московская обл., г. Люберцы, ул. Авиат...",5027183777.00,Московская область,Строительство жилых и нежилых зданий,Юридическое лицо признано несостоятельным (бан...,"""STR""",2021-06-01,...,148.01,0.08,0.04,0.98,0.04,141395000.00,10373000.00,0.35,True,2019.00
77236,77236,"ЯРАВТОДОР, ООО",1147602008520.00,"150506, Ярославская обл., Ярославский район, п...",7602110885.00,Ярославская область,Строительство жилых и нежилых зданий,\nСообщение о признаках банкротства (17.10.202...,"""STR""",2020-11-02,...,724.22,0.01,0.06,0.17,0.06,228319000.00,-12804000.00,1.42,True,2018.00
